# Processing NYT Election data

The goal of this notebook is to process the JSONs received from the NYT APIs into CSVs that will be easier to work with.

In [48]:
import pandas as pd
import json
from collections import defaultdict
import pprint

## House data

In [72]:
with open('house_11-16.json') as f:
  house_json = json.load(f)

In [77]:
# for one house race, tally vote results
random_race = house_json['data']['races'][0]
race_results = {'democrat': 0, 'republican': 0}
opposite_party = {'democrat': 'republican', 'republican': 'democrat'}
for c in random_race['counties']:
    votes = c['results'].values()
    if(len(votes) > 2):
        print("Warning...something spooky...")
        print(c)
        print("")
    max_votes = max(votes)
    min_votes = min(votes)
    race_results[c['leader_party_id']] += max_votes
    race_results[opposite_party[c['leader_party_id']]] += min_votes
    
print(race_results)
print(random_race['leader_margin_votes'])
print(race_results[random_race['leader_party_id']] - race_results[opposite_party[random_race['leader_party_id']]])
print(random_race.keys())
print(random_race['state_id'])
print(random_race['seat'])
pprint.pprint(random_race['candidates'])

{'democrat': 156754, 'republican': 188154}
31400
31400
dict_keys(['race_id', 'race_slug', 'url', 'state_page_url', 'ap_polls_page', 'race_type', 'election_type', 'election_date', 'runoff', 'race_name', 'office', 'officeid', 'nyt_race_description', 'race_rating', 'nyt_key_race', 'seat', 'seat_name', 'state_id', 'state_slug', 'state_name', 'state_nyt_abbrev', 'state_shape', 'state_aspect_ratio', 'party_id', 'uncontested', 'report', 'result', 'result_source', 'gain', 'lost_seat', 'votes', 'absentee_votes', 'absentee_counties', 'absentee_count_progress', 'absentee_outstanding', 'absentee_max_ballots', 'provisional_outstanding', 'provisional_count_progress', 'poll_display', 'poll_countdown_display', 'poll_waiting_display', 'poll_time', 'poll_time_short', 'precincts_reporting', 'precincts_total', 'reporting_display', 'reporting_value', 'eevp', 'tot_exp_vote', 'eevp_source', 'eevp_value', 'eevp_display', 'county_data_source', 'incumbent_party', 'no_forecast', 'last_updated', 'candidates', 'ha

In [78]:
party_ids = set()
for r in house_json['data']['races']:
    for c in r['candidates']:
        party_ids.add(c['party_id'])
            
print(party_ids)

{'democrat', 'republican', 'other'}


In [79]:
results = defaultdict(list)
# keys: state_id, seat, d_votes, r_votes, other_votes, d_vote_share, r_vote_share, d_winner, r_winner, STATEFP, CD114FP

state_codes = {
    'WA': '53', 'DE': '10', 'DC': '11', 'WI': '55', 'WV': '54', 'HI': '15',
    'FL': '12', 'WY': '56', 'PR': '72', 'NJ': '34', 'NM': '35', 'TX': '48',
    'LA': '22', 'NC': '37', 'ND': '38', 'NE': '31', 'TN': '47', 'NY': '36',
    'PA': '42', 'AK': '02', 'NV': '32', 'NH': '33', 'VA': '51', 'CO': '08',
    'CA': '06', 'AL': '01', 'AR': '05', 'VT': '50', 'IL': '17', 'GA': '13',
    'IN': '18', 'IA': '19', 'MA': '25', 'AZ': '04', 'ID': '16', 'CT': '09',
    'ME': '23', 'MD': '24', 'OK': '40', 'OH': '39', 'UT': '49', 'MO': '29',
    'MN': '27', 'MI': '26', 'RI': '44', 'KS': '20', 'MT': '30', 'MS': '28',
    'SC': '45', 'KY': '21', 'OR': '41', 'SD': '46'
}

for r in house_json['data']['races']:
    if r['leader_margin_votes'] is None or len(r['leader_party_id']) == 0:
        # skip if the data is insufficient
        continue
    race_results = {'democrat': 0, 'republican': 0, 'other': 0}
    winning_party = {'d_won': 0, 'r_won': 0, 'other_won': 0}
    for c in r['candidates']:
        if c['party_id'] not in ['democrat', 'republican']:
            c['party_id'] = 'other'
        race_results[c['party_id']] += c['votes']
        if c['winner']:
            if c['party_id'] == 'democrat':
                winning_party['d_won'] = 1
            elif c['party_id'] == 'republican':
                winning_party['r_won'] = 1
            else:
                winning_party['other_won'] = 1
                
    results['state_id'].append(r['state_id'])
    results['STATEFP'].append(state_codes[r['state_id']])
    results['seat_id'].append(r['seat'])
    # copied from seat_id
    results['CD114FP'].append(r['seat'])
    
    # vote counts
    results['d_votes'].append(race_results['democrat'])
    results['r_votes'].append(race_results['republican'])
    results['other_votes'].append(race_results['other'])
    # vote shares
    total_votes = sum(race_results.values())
    results['d_vote_share'].append(race_results['democrat'] / total_votes)
    results['r_vote_share'].append(race_results['republican'] / total_votes)
    results['other_vote_share'].append(race_results['other'] / total_votes)
    
    for k in winning_party.keys():
        results[k].append(winning_party[k])

In [80]:
house_df = pd.DataFrame.from_dict(results)
house_df.head()

,state_id,STATEFP,seat_id,CD114FP,d_votes,r_votes,other_votes,d_vote_share,r_vote_share,other_vote_share,d_won,r_won,other_won
0,AK,02,1,1,156754,188154,0,0.454481,0.545519,0.000000,0,1,0
1,AL,01,1,1,96363,204399,0,0.320396,0.679604,0.000000,0,1,0
2,AL,01,2,2,104592,197329,286,0.346094,0.652960,0.000946,0,1,0
3,AL,01,3,3,103051,214731,0,0.324282,0.675718,0.000000,0,1,0
4,AL,01,4,4,50730,246668,0,0.170579,0.829421,0.000000,0,1,0


In [83]:
# some sanity checks
print(house_df['d_won'].sum())
print(house_df['r_won'].sum())
print(house_df['other_won'].sum())

215
199
0


In [82]:
house_df.to_csv("clean_csv/house.csv")

# Senate data

In [84]:
with open('senate_11-16.json') as f:
  senate_json = json.load(f)

In [86]:
random_race = senate_json['data']['races'][0]
print(random_race.keys())
pprint.pprint(random_race['candidates'])

dict_keys(['race_id', 'race_slug', 'url', 'state_page_url', 'ap_polls_page', 'race_type', 'election_type', 'election_date', 'runoff', 'race_name', 'office', 'officeid', 'nyt_race_description', 'race_rating', 'seat', 'seat_name', 'state_id', 'state_slug', 'state_name', 'state_nyt_abbrev', 'state_shape', 'state_aspect_ratio', 'party_id', 'uncontested', 'report', 'result', 'result_source', 'gain', 'lost_seat', 'votes', 'absentee_votes', 'absentee_counties', 'absentee_count_progress', 'absentee_outstanding', 'absentee_max_ballots', 'provisional_outstanding', 'provisional_count_progress', 'poll_display', 'poll_countdown_display', 'poll_waiting_display', 'poll_time', 'poll_time_short', 'precincts_reporting', 'precincts_total', 'reporting_display', 'reporting_value', 'eevp', 'tot_exp_vote', 'eevp_source', 'eevp_value', 'eevp_display', 'county_data_source', 'incumbent_party', 'no_forecast', 'last_updated', 'candidates', 'has_incumbent', 'leader_margin_value', 'leader_margin_votes', 'leader_mar

In [87]:
results = defaultdict(list)
# keys: state_id, d_votes, r_votes, other_votes, d_vote_share, r_vote_share, d_winner, r_winner, STATEFP, CD114FP

for r in senate_json['data']['races']:
    if r['leader_margin_votes'] is None or len(r['leader_party_id']) == 0:
        # skip if the data is insufficient
        continue
    race_results = {'democrat': 0, 'republican': 0, 'other': 0}
    winning_party = {'d_won': 0, 'r_won': 0, 'other_won': 0}
    for c in r['candidates']:
        if c['party_id'] not in ['democrat', 'republican']:
            c['party_id'] = 'other'
        race_results[c['party_id']] += c['votes']
        if c['winner']:
            if c['party_id'] == 'democrat':
                winning_party['d_won'] = 1
            elif c['party_id'] == 'republican':
                winning_party['r_won'] = 1
            else:
                winning_party['other_won'] = 1
                
    results['state_id'].append(r['state_id'])
    results['STATEFP'].append(state_codes[r['state_id']])
    
    # vote counts
    results['d_votes'].append(race_results['democrat'])
    results['r_votes'].append(race_results['republican'])
    results['other_votes'].append(race_results['other'])
    # vote shares
    total_votes = sum(race_results.values())
    results['d_vote_share'].append(race_results['democrat'] / total_votes)
    results['r_vote_share'].append(race_results['republican'] / total_votes)
    results['other_vote_share'].append(race_results['other'] / total_votes)
    
    for k in winning_party.keys():
        results[k].append(winning_party[k])

In [88]:
senate_df = pd.DataFrame.from_dict(results)
senate_df.head()

,state_id,STATEFP,d_votes,r_votes,other_votes,d_vote_share,r_vote_share,other_vote_share,d_won,r_won,other_won
0,AK,02,143141,187471,16219,0.412711,0.540526,0.046763,0,1,0
1,AL,01,913999,1385344,3869,0.396837,0.601483,0.001680,0,1,0
2,AR,05,0,790638,395932,0.000000,0.666322,0.333678,0,1,0
3,AZ,04,1716467,1637661,594,0.511657,0.488166,0.000177,1,0,0
4,CO,08,1730722,1429085,75001,0.535031,0.441784,0.023186,1,0,0


In [93]:
print(senate_df.shape)
# some sanity checks
print(senate_df['d_won'].sum())
print(senate_df['r_won'].sum())
print(senate_df['other_won'].sum())

(35, 11)
13
20
0


In [94]:
senate_df.to_csv("clean_csv/senate.csv")

## Presidential results

In [95]:
with open('president_11-16.json') as f:
  president_json = json.load(f)

In [96]:
random_race = president_json['data']['races'][0]
print(random_race.keys())
pprint.pprint(random_race['candidates'])

dict_keys(['race_id', 'race_slug', 'url', 'state_page_url', 'ap_polls_page', 'race_type', 'election_type', 'election_date', 'runoff', 'race_name', 'office', 'officeid', 'race_rating', 'seat', 'seat_name', 'state_id', 'state_slug', 'state_name', 'state_nyt_abbrev', 'state_shape', 'state_aspect_ratio', 'party_id', 'uncontested', 'report', 'result', 'result_source', 'gain', 'lost_seat', 'votes', 'electoral_votes', 'absentee_votes', 'absentee_counties', 'absentee_count_progress', 'absentee_outstanding', 'absentee_max_ballots', 'provisional_outstanding', 'provisional_count_progress', 'poll_display', 'poll_countdown_display', 'poll_waiting_display', 'poll_time', 'poll_time_short', 'precincts_reporting', 'precincts_total', 'reporting_display', 'reporting_value', 'eevp', 'tot_exp_vote', 'eevp_source', 'eevp_value', 'eevp_display', 'county_data_source', 'incumbent_party', 'no_forecast', 'last_updated', 'candidates', 'has_incumbent', 'leader_margin_value', 'leader_margin_votes', 'leader_margin_d

In [97]:
results = defaultdict(list)
# keys: state_id, d_votes, r_votes, other_votes, d_vote_share, r_vote_share, d_winner, r_winner, STATEFP, CD114FP

for r in president_json['data']['races']:
    if r['leader_margin_votes'] is None or len(r['leader_party_id']) == 0:
        # skip if the data is insufficient
        continue
    race_results = {'democrat': 0, 'republican': 0, 'other': 0}
    winning_party = {'d_won': 0, 'r_won': 0, 'other_won': 0}
    for c in r['candidates']:
        if c['party_id'] not in ['democrat', 'republican']:
            c['party_id'] = 'other'
        race_results[c['party_id']] += c['votes']
        if c['winner']:
            if c['party_id'] == 'democrat':
                winning_party['d_won'] = 1
            elif c['party_id'] == 'republican':
                winning_party['r_won'] = 1
            else:
                winning_party['other_won'] = 1
                
    results['state_id'].append(r['state_id'])
    results['STATEFP'].append(state_codes[r['state_id']])
    
    # vote counts
    results['d_votes'].append(race_results['democrat'])
    results['r_votes'].append(race_results['republican'])
    results['other_votes'].append(race_results['other'])
    # vote shares
    total_votes = sum(race_results.values())
    results['d_vote_share'].append(race_results['democrat'] / total_votes)
    results['r_vote_share'].append(race_results['republican'] / total_votes)
    results['other_vote_share'].append(race_results['other'] / total_votes)
    
    for k in winning_party.keys():
        results[k].append(winning_party[k])

In [98]:
prez_df = pd.DataFrame.from_dict(results)
prez_df.head()

,state_id,STATEFP,d_votes,r_votes,other_votes,d_vote_share,r_vote_share,other_vote_share,d_won,r_won,other_won
0,AK,02,150262,185769,13442,0.429967,0.531569,0.038464,0,1,0
1,AL,01,843473,1434159,32268,0.365156,0.620875,0.013969,0,1,0
2,AR,05,419863,757833,34288,0.346426,0.625283,0.028291,0,1,0
3,AZ,04,1672143,1661686,52016,0.493863,0.490774,0.015363,1,0,0
4,CA,06,10761303,5751655,361985,0.637709,0.340840,0.021451,1,0,0


In [100]:
print(prez_df.shape)
# some sanity checks
print(prez_df['d_won'].sum()) # :')
print(prez_df['r_won'].sum())
print(prez_df['other_won'].sum())

(51, 11)
26
25
0


In [94]:
senate_df.to_csv("clean_csv/senate.csv")